In [2]:
# Imports
from classes.cbnetwork import CBN
from classes.experiment import ExperimentModel

import ray
import time
import pandas as pd
import numpy as np
import pickle  # library to serialization object

from itertools import product  # generate combinations of numbers

# Ray Configurations
# ray.shutdown()
#runtime_env = {"working_dir": "/home/reynaldo/Documents/RESEARCH/SynEstRDDA", "pip": ["requests", "pendulum==2.1.2"]}
#ray.init(address='ray://172.17.163.253:10001', runtime_env=runtime_env, log_to_driver=False)
# ray.init(address='ray://172.17.163.244:10001', runtime_env=runtime_env , log_to_driver=False, num_cpus=12)
# ray.init(log_to_driver=False, num_cpus=12)

ModuleNotFoundError: No module named 'igraph'

In [ ]:
# Parameters
n_rdds = 5
n_of_variables_rdda = 5
n_of_signals_rdda = 2
n_exit_variables = 2
n_clauses_function = 2
type_network = "ALEATORY"

# Generate the RDDA
o_rdda = RedRddasModel(number_of_rddas=n_rdds,number_of_variables_rdda=n_of_variables_rdda,number_of_signals_rdda=n_of_signals_rdda,number_exit_variables=n_exit_variables,number_clauses_function=n_clauses_function)

# Show the RDDA
o_rdda.show_description()

# Generate the RDDs
o_rdda.generate_rddas(type_network=type_network)

# Show the list of RDDs
o_rdda.show_rdds()

# Find the attractors by RDD
o_rdda = RedRddasModel.find_attractors_rddas(o_rdda)

# Show the attractors of the RDDs by Signal
o_rdda.show_attractors()

# Calculation the Attractor Pairs
o_rdda = RedRddasModel.calculation_compatible_pairs(o_rdda)

# Show the list of attractor pairs
o_rdda.show_attractor_pairs()

# Assembly the attractor fields
l_partial_paths = o_rdda.assembly_attractor_fields_pruning(o_rdda)

# Show the list of attractor fields
o_rdda.show_attractor_fields_detail()

In [ ]:
# # partial_solutions = [ iteration = [] ]
# for iteration in l_partial_solutions:
#     print("Number of Partial Solutions:", len(iteration))
#     for partial_solution in iteration:
#         print("Part :", partial_solution)

# print(o_rdda.list_signal_pairs)
# for l_pairs in o_rdda.list_signal_pairs:
#     print("Number of pairs from list :", len(l_pairs))

# Assembly the attractor fields and save the partial solutions
df_exec_experiment = o_rdda.assembly_attractor_fields_experiment()
df_exec_experiment

In [ ]:
# Indicators about execution of the method
df_exec_experiment['iterative_after'] = df_exec_experiment['base_list_iterative'] * df_exec_experiment['second_list_pairs']
df_exec_experiment['percent_methods'] = 100 - ( (df_exec_experiment['base_list_after'] * 100) / df_exec_experiment['iterative_after'])
df_exec_experiment

### Graph the values of every method

In [ ]:
import matplotlib.pyplot as plt
df_exec_experiment['id'] = range(1, len(df_exec_experiment) + 1)
df_exec_experiment.set_index('id')
df_exec_experiment

In [ ]:
import matplotlib.pyplot as plt
x_axs = list(df_exec_experiment['id'])
y_axs_enumerate = list(df_exec_experiment['iterative_after'])
y_axs_pruning = list(df_exec_experiment['base_list_after'])

fig, ax = plt.subplots()
ax.plot(x_axs, y_axs_enumerate, label="Enumerative")
ax.plot(x_axs, y_axs_pruning, label="Pruning")
ax.legend()
plt.xlabel("Iteration")
plt.ylabel("Candidates")
plt.title("Benchmark beetwen Methods")
plt.savefig('data/exp_benchmark_enumerative_pruning_fig.eps')
plt.show()

### Generate the latex table with pandas

In [ ]:
df = df_exec_experiment[['first_list_pairs','second_list_pairs', 'base_list_after', 'iterative_after']].copy().\
    rename(columns={'first_list_pairs': 'Lista Base', 'second_list_pairs': 'Lista Adicionada', 'base_list_after': 'Precandidatos Podas', 'iterative_after' : 'Precandidatos Enumerativo'})
df

In [ ]:
# Export Dataframe to Latex a
print(df.to_latex(index=False))

In [ ]:
# Tree pruning in the Assembly of attractor fields
# imports
import matplotlib.pyplot as plt
import networkx as nx
import pydot
from networkx.drawing.nx_pydot import graphviz_layout
import numpy as np

# Create Dictionary
d_pairs = {}
v_key = 1
for v_pair in o_rdda.list_attractors_pairs:
    d_pairs[v_key] = v_pair
    v_key = v_key + 1

# Show the dictionary and the
for v_key, v_value in d_pairs.items():
    print(v_key, ":", v_value)

# List all the partial paths in the assembly attractor fields method
l_path_edge = []
for l_path in l_partial_paths:
    for path in l_path:
        # print(path)
        # select the final edges
        l_path_edge.append(path[-2:])
        # print(path[-2:])

# fill the edges with dict
l_edges_dict = []
for v_edge in l_path_edge:
    v_edge_dict = []
    for key, value in d_pairs.items():
        if value == v_edge[0]:
            v_edge_dict.append(key)
            # print(value)
            # print(v_edge[0])
            # print(key)
    for key, value in d_pairs.items():
        if value == v_edge[1]:
            v_edge_dict.append(key)
            # print(value)
            # print(v_edge[1])
            # print(key)
    l_edges_dict.append(v_edge_dict)
print(l_edges_dict)

In [ ]:
# Tree pruning in the Assembly of attractor fields
# imports
import matplotlib.pyplot as plt
import networkx as nx
import pydot
from networkx.drawing.nx_pydot import graphviz_layout
import numpy as np

# Generate the graph with the paths
T = nx.Graph()
# T.add_nodes_from(d_pairs.keys())
T.add_edges_from(l_edges_dict)

# Generate the Layout with dot
pos = graphviz_layout(T, prog="dot")
# Fix the position problem, string to int
aux_pos ={}
for key in pos.keys():
    aux_pos[int(key)] = pos[key]
pos = aux_pos
# Generate the graph with the data an information
nx.draw_networkx(T, pos, font_size=10)
# Show the graph with parameters
plt.figure(figsize=(1200,600), dpi=900)
plt.show()

# Generate full tree for enumerate method

In [ ]:
# Generate the edges
l_aux = []
v_begin = 1
for element in o_rdda.list_signal_pairs:
    l_aux.append(list(range(v_begin , v_begin + len(element))))
    v_begin = v_begin + len(element)
# for element in l_aux:
#     print(element)

l_path_edge = []
for v_index in range(0,len(l_aux)-1):
    l_path_edge = l_path_edge + list(product(l_aux[v_index],l_aux[v_index+1]))
    # print(l_path_edge)

# Generate the graph with the paths
T = nx.Graph()
T.add_edges_from(l_path_edge)

# Generate the Layout with dot
pos = graphviz_layout(T, prog="dot")
# Fix the position problem, string to int
aux_pos ={}
for key in pos.keys():
    aux_pos[int(key)] = pos[key]
pos = aux_pos
# Generate the graph with the data an information
nx.draw_networkx(T, pos, font_size=10)
# Show the graph with parameters
plt.figure(figsize=(1200,600), dpi=900)
plt.show()